import pandas as pd

In [1]:
import pandas as pd

In [2]:
tripadv_review = pd.read_csv(r'C:\Users\Dell\Desktop\Restaurant\The Gym.csv', error_bad_lines=False);

In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [6]:
processed_reviews = tripadv_review['r_text'].map(preprocess)
processed_reviews

0      [highly, recommended, elegant, environment, ni...
1      [went, whim, buns, wound, best, tasted, emeral...
2      [paid, euros, miserable, buns, probably, days,...
3      [professional, respectful, kind, staff, feel, ...
4      [definitely, suggested, amazing, cocktails, fo...
                             ...                        
226    [visited, saturday, quick, lunch, chose, table...
227    [great, casual, atmosphere, great, food, good,...
228    [went, couple, times, drinks, dinner, popular,...
229    [excited, downs, norm, impressive, menu, limit...
230    [nicosia, food, scene, pretty, uninspiring, ex...
Name: r_text, Length: 231, dtype: object

In [7]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]]


In [8]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=250,
                                           passes=10,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 100,
                                           eval_every = None)

In [9]:
from pprint import pprint

In [10]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.015*"food" + 0.011*"nice" + 0.011*"wine" + 0.010*"cocktails" + '
  '0.009*"service" + 0.009*"menu" + 0.008*"place" + 0.007*"restaurant" + '
  '0.007*"mustard" + 0.007*"salad"'),
 (1,
  '0.024*"good" + 0.019*"place" + 0.011*"food" + 0.010*"service" + '
  '0.010*"cheese" + 0.009*"nice" + 0.008*"salad" + 0.008*"lunch" + '
  '0.007*"salmon" + 0.006*"sandwich"'),
 (2,
  '0.032*"good" + 0.021*"service" + 0.021*"cocktails" + 0.019*"food" + '
  '0.016*"nice" + 0.014*"great" + 0.014*"staff" + 0.014*"atmosphere" + '
  '0.013*"place" + 0.011*"friendly"'),
 (3,
  '0.016*"place" + 0.015*"cocktails" + 0.012*"service" + 0.011*"good" + '
  '0.011*"food" + 0.011*"menu" + 0.010*"dinner" + 0.009*"dishes" + '
  '0.008*"evening" + 0.008*"great"'),
 (4,
  '0.015*"place" + 0.015*"glass" + 0.015*"wine" + 0.013*"excellent" + '
  '0.013*"best" + 0.009*"food" + 0.009*"time" + 0.009*"minutes" + '
  '0.009*"recommend" + 0.007*"table"'),
 (5,
  '0.025*"food" + 0.020*"place" + 0.015*"great" + 0.015*"best" 

In [11]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.068193  0.020632       1        1  16.800138
6      0.059561  0.048251       2        1  15.439275
7      0.066624 -0.004296       3        1  13.515808
2      0.071783  0.028301       4        1  12.661799
1      0.030680 -0.164307       5        1  10.346688
3     -0.019931  0.055954       6        1  10.254822
8     -0.029091 -0.006166       7        1   6.235538
0     -0.124600 -0.017610       8        1   6.184010
4     -0.047779  0.025919       9        1   4.733785
5     -0.075440  0.013323      10        1   3.828134, topic_info=          Term       Freq      Total Category  logprob  loglift
15        best  20.000000  20.000000  Default  30.0000  30.0000
276      glass  10.000000  10.000000  Default  29.0000  29.0000
126       wine  27.000000  27.000000  Default  28.0000  28.0000
46      dinner  32.000000  32.000000  Default  27.0000  27.0000
77   cocktails  56.000000  56.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
7         menu   1.214862  48.071350  Topic10  -5.2585  -0.4153
12     service   1.215226  79.633575  Topic10  -5.2582  -0.9197
448   pancakes   0.666598   4.680555  Topic10  -5.8587   1.3138
457      times   0.682965  14.103526  Topic10  -5.8344   0.2351
117      salad   0.687581  21.191628  Topic10  -5.8277  -0.1654

[735 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
956       3  0.634605    accommodate
1680      5  0.652763   accomodating
13        9  0.725073  accompaniment
334       6  0.652113   accomplished
306       1  0.131095       actually
...     ...       ...            ...
228       6  0.090363          worth
228      10  0.090363          worth
24        9  0.725073          wound
458      10  0.523745           wrap
299       2  0.821992           yard

[1350 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 7, 8, 3, 2, 4, 9, 1, 5, 6])